# Can we append a different phenomenon onto a zarr with the same dims

In [2]:
import iris
import os
import xarray as xr
import numpy as np

In [10]:
def pp_to_cube(filename, filepath, constraints={}):
    # Load a cube from a .pp file
    cube, = iris.load(os.path.join(filepath, filename), iris.AttributeConstraint(**constraints))
    print(f'Cube loaded from {filename}')
    return cube

In [4]:
def cube_to_xr(cube):
    # Convert Iris cube to Xarray Dataset
    return xr.DataArray.from_iris(cube).to_dataset()

In [44]:
def xr_to_zarr(dataset, zarr_store, chunks={'time':10, 'grid_latitude':219, 'grid_longitude':286}, append_dim='time'):
    # Write dataset to new zarr store
    # OR append dataset to an existing zarr store
    dataset = dataset.chunk(chunks=chunks)
    if os.path.isdir(zarr_store):
        dataset.to_zarr(zarr_store, consolidated=True, append_dim=append_dim)
        print(f'Appended cube to {zarr_store}')
    else:
        dataset.to_zarr(zarr_store, mode='w', consolidated=True)
        print(f'Written cube to {zarr_store}')

In [6]:
def datetimes_from_cube(cube):
    return xr.DataArray.from_iris(cube).time.data

def datetimes_from_zarr(zarr_store):
    return xr.open_zarr(zarr_store).time.data

## Can we add a new phenomenon to a Dataset that is only a subset of the domain?

In [36]:
# Load surface_air_pressure data
STASH = 'm01s00i001'
filepath = '/data/cssp-china/mini-dataset-24-01-19/20CR/daily'
files = sorted(os.listdir(filepath))

cube = pp_to_cube(files[3], filepath, constraints={'STASH': STASH})
# cubelist = iris.load(os.path.join(filepath, files[1]))
cube

Cube loaded from apepda.pa51240.pp


<iris 'Cube' of surface_air_pressure / (Pa) (time: 10; grid_latitude: 219; grid_longitude: 286)>

In [37]:
# Load zarr and see if we can smüsch another DataArray onto it
ds_z = xr.open_zarr(zarr_store)
ds_y = cube_to_xr(cube)

print(ds_z)
print('---'*20)
print(ds_y)

<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 240)
Coordinates:
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
    forecast_reference_time  datetime64[ns] ...
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
  * time                     (time) datetime64[ns] 1851-01-05T12:00:00 ... 1851-09-01T12:00:00
Data variables:
    precipitation_flux       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(10, 219, 286), meta=np.ndarray>
------------------------------------------------------------
<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 10)
Coordinates:
  * time                     (time) datetime64[ns] 1851-01-25T12:00:00 ... 1851-02-03T12:00:00
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  *

In [38]:
ds_new = xr.merge([ds_z, ds_y])

In [39]:
ds_new.surface_air_pressure.data.compute()

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

### Yes! We can, Xarray gracefully fills the missing data with NaNs

## Can we naively append a new phenomenon to a Zarr that aligns to a subset of the domain?

In [40]:
ds_y

<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 10)
Coordinates:
  * time                     (time) datetime64[ns] 1851-01-25T12:00:00 ... 1851-02-03T12:00:00
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
    forecast_reference_time  datetime64[ns] ...
    forecast_period          (time) timedelta64[ns] ...
Data variables:
    surface_air_pressure     (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 219, 286), meta=np.ndarray>

In [45]:
%%time
zarr_store = '/data/cssp-china/zarr_precip_append'
xr_to_zarr(ds_y, zarr_store)

Appended cube to /data/cssp-china/zarr_precip_append
CPU times: user 118 ms, sys: 33.1 ms, total: 151 ms
Wall time: 3.58 s


In [46]:
xr.open_zarr(zarr_store)

ValueError: conflicting sizes for dimension 'time': length 240 on 'precipitation_flux' and length 250 on 'forecast_period'

### OK, it's not done a very good job

## Can we load a zarr, merge in another Dataset, then save back to a zarr?

In [47]:
%%time
zarr_store_new = '../zarr_multiphenom'
xr_to_zarr(ds_new, zarr_store_new)

Written cube to ../zarr_multiphenom
CPU times: user 638 ms, sys: 213 ms, total: 851 ms
Wall time: 3.97 s


In [51]:
ds_z_new = xr.open_zarr(zarr_store_new)
ds_z_new

<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 240)
Coordinates:
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
    forecast_reference_time  datetime64[ns] ...
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
  * time                     (time) datetime64[ns] 1851-01-05T12:00:00 ... 1851-09-01T12:00:00
Data variables:
    precipitation_flux       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(10, 219, 286), meta=np.ndarray>
    surface_air_pressure     (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(10, 219, 286), meta=np.ndarray>

In [50]:
ds_z_new.surface_air_pressure.data.compute()

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [52]:
ds_z_new.precipitation_flux.data.compute()

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.35184362e-04, 3.58784921e-04, 3.65556130e-04, ...,
         3.75007716e-04, 3.48673697e-04, 3.43388354e-04],
        [3.38655518e-04, 3.62407154e-04, 3.77864257e-04, ...,
         3.04952788e-04, 3.36193130e-04, 3.27009853e-04],
        ...,
        [2.94131332e-05, 2.19907688e-05, 2.09703267e-05, ...,
         3.10555350e-07, 3.08261292e-07, 4.10253961e-07],
        [3.41830491e-05, 2.92384539e-05, 2.81920620e-05, ...,
         2.55267736e-07, 4.26608466e-07, 5.71437454e-07],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.35056888e-04, 3.18218285e-04, 3.29081959e-04, ...,
         1.91064959e-04, 1.84633216e-04, 1.47610874e-04],
        [3.21774976e-04, 

In [53]:
ds_z_new.surface_air_pressure.data

dask.array<zarr, shape=(240, 219, 286), dtype=float32, chunksize=(10, 219, 286), chunktype=numpy.ndarray>

# CONCLUSION
### If we want to add multiple phenomena to a zarr, we need to combine them into a full dataset before saving to a Zarr
Therefore a strategy could be:
1. Load single .pp files with Iris
2. Pull out all phenomena for that time step
3. Convert to Xarray Dataset (cube-wise arrays then merge into a Dataset?)
4. Append to a Zarr